In [1]:
import sys

# Specify the directory you want to add
custom_path = './..'

# Add the directory to sys.path
if custom_path not in sys.path:
    sys.path.append(custom_path)

In [2]:
from __future__ import absolute_import
import importlib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from models import MLP, OneOverSqr
from runners import wrap_runner_for_optimization
from fitting_utils import create_fitting_loss

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


def scale_and_bias(x,bias=0,scale=45.0):
    x = np.array(x)
    return x*scale + bias
#to sweep
fixed_params={}
fixed_params['model'] =  dict(n_inputs = 3,
        n_hidden = 5*4*512,
        n_outs = 1,
        en_bias = False,
        b_high=3, first_layer_init='uniform_unity',
        first_layer_weights_trainable = True,
        out_layer_init='zeros')         

fixed_params['runner'] = {'criterion':'MSE', 'k':[0,1,0], 'sigma_noi':0.0, 'tau_u':1,
                        'save_model_at_init':False, 'ic_param_file':None}

optim_param_mapping= [('custom','normalized_log_lr'),
                    ('model','skip_gain'),                      
                    ('custom','w'),
                    ('postprocessing','scale'),
                    ('custom', 'k1'),
                    ('custom', 'k3'),
                    ]

custom_param_mappings = [{'cathegory':'runner','param_name':'learning_rate',
                        'fun': lambda x: 10.**x['normalized_log_lr']/x['w']},
                        {'cathegory':'runner','param_name':'k',
                        'fun': lambda x: [x['k1'],0,x['k3']]},
                        {'cathegory':'model','param_name':'nl','fun': lambda x: (lambda : OneOverSqr(w=x['w']))}]



wrapped_runner = wrap_runner_for_optimization(model_class=MLP, 
                                             fixed_params=fixed_params,
                                             optim_params_mapping=optim_param_mapping,
                                             custom_param_mappings=custom_param_mappings, 
                                             postprocessing_fun=scale_and_bias)


In [4]:
df = pd.read_csv('../../../frogs_project/data/avraham__ivry_fig1_rotFig1_v2.csv',header=None)
df = df.T
data = df.to_numpy()

In [5]:
stimulus = [(1,40),(0,41)]
datapoint_mapping={'data':lambda x:x, 'model_output': lambda x:x[1:]} #a hack that mitigates the fact that first perturbatrion datapoint is 'responsive'

pooling_funs = ['MSE'] #,'MAE']

In [6]:
fitting_loss = {pooling_fun : create_fitting_loss(data=data[:80,0],stimulus=stimulus,wrapped_model=wrapped_runner,
                    pooling_fun=pooling_fun,
                    datapoint_mapping=datapoint_mapping,
                    weighting=None) for pooling_fun in pooling_funs}


In [7]:
from scipy.optimize import minimize, basinhopping

In [8]:
# x0 = [-4.5,0.4,0.5, 40]
# bounds = [(-5.5,-4),(-0.9,0.99), (0.05,3), (10,90)]
# # bounds = [(-5.5,-4),(-0.9,0.99), (0.4,3), (20,50)]

x0 = [-4.5,0.4,0.5, 40, 1, 1]
bounds = [(-6,-3),(-0.9,0.99), (0.05,3), (10,90), (0,2), (0,2)]

In [9]:
out = wrapped_runner([(0,40),(1,40)]*3,x0)

In [10]:
opt_out = {}

def print_fun(x, f, accepted):
        print("at minimum %.4f accepted %d  with params %s" % (f, int(accepted), str(x)))

In [11]:
# res = minimize(fitting_loss, x0, method='nelder-mead', bounds= bounds,
#                options={'xatol': 1e-6, 'disp': True})

# def print_fun(x, f, accepted):
#         print("at minimum %.4f accepted %d" % (f, int(accepted)))
# res = basinhopping(fitting_loss, x0, 
#                    minimizer_kwargs = dict(method='nelder-mead', options={'xatol': 1e-6, 'disp': True, 'bounds':bounds}),
#                    callback=print_fun)
# 




# res = basinhopping(fitting_loss, x0, 
#                    minimizer_kwargs = dict(method='L-BFGS-B', bounds= bounds),
#                    callback=print_fun)
# method='L-BFGS-B', bounds= bounds,

In [12]:
# opt_out['MSE'] = res

In [13]:
pooling_fun = pooling_funs[-1]
opt_out[pooling_fun] = basinhopping(fitting_loss[pooling_fun], x0, niter=2,
                   minimizer_kwargs = dict(method='L-BFGS-B', bounds= bounds),
                   callback=print_fun)

at minimum 299.6026 accepted 1  with params [-4.49999994  0.40000002  0.50000009 40.00000184  1.00000004  1.00000004]
at minimum 386.6875 accepted 0  with params [-4.24576374  0.49907891  0.97641212 40.35401086  1.08278534  0.77422177]
at minimum 274.5158 accepted 1  with params [-4.95440679  0.41199707  0.18597428 40.25998328  1.2820405   1.41543078]


In [ ]:
opt_out[(pooling_fun,'nelder-mead')] = basinhopping(fitting_loss[pooling_fun], x0, 
                   minimizer_kwargs = dict(method='nelder-mead', bounds= bounds),
                   callback=print_fun)

/opt/conda/lib/python3.8/site-packages/scipy/optimize/_minimize.py:535: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  warn('Method %s cannot handle constraints nor bounds.' % method,


at minimum 220.3084 accepted 1  with params [-4.67669815e+00  1.14252062e-02  9.08974528e-01  3.51182078e+01
  1.11279349e+00 -9.71994849e-02]
at minimum 223.2524 accepted 0  with params [-4.7943921   0.37579662  0.52253953 35.12008081  0.60919124  0.27790259]
at minimum 223.3672 accepted 1  with params [-4.70382767  0.12422999  0.6260617  34.58722997  1.37312122  0.37098691]


In [ ]:
fitting_loss['MAE'](x0), fitting_loss['MSE'](x0)

In [ ]:
res

In [ ]:
opt_out

In [ ]:
outs  = {pooling_fun: wrapped_runner([(1,40),(0,40),(1,40)],opt_out[pooling_fun]['x']) for pooling_fun in pooling_funs}

In [ ]:
appendix = ('MAE',  'nelder-mead')
outs[appendix] = wrapped_runner([(1,40),(0,40),(1,40)],opt_out[appendix]['x'])

In [ ]:
for setting in pooling_funs +[appendix]:
    plt.plot(outs[setting])
plt.plot(data[:,0],'.')
plt.plot(data[:80,0],'.')
plt.plot(smooth_signal(data[:,0],3))


In [ ]:
for setting in pooling_funs +[appendix]:
    plt.plot(outs[setting])
plt.plot(data[:,0],'x')
plt.plot(data[:80,0],'x')

plt.xlim([39,120])
plt.ylim([-5,60])


In [ ]:
mae = lambda x: np.mean(np.abs(x))
mse = lambda x: np.mean(np.square(x))


In [ ]:
for setting in pooling_funs +[appendix]:
    fit_err = outs[setting][:80] - data[:80,0]
    val_err = outs[setting][80:] - data[80:,0]
    print(f'fit objective: {setting}')
    print(f' fitting MAE: {mae(fit_err)}; MSE: {mse(fit_err)}')
    print(f' validation MAE: {mae(val_err)}; MSE: {mse(val_err)}')

In [ ]:
import scipy
scipy.__version__

In [ ]:
# !pip3 install --upgrade pip

In [ ]:
# pip install --upgrade scipy

In [ ]:
from dsp_utils import smooth_signal

In [ ]:
import pickle

In [ ]:
with open('sweep_1_subj_12.pkl','rb') as f:
    opt_data = pickle.load(f)

In [ ]:
opt_data